In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:


# # Google Apps Script Web App URL
# url = "https://script.google.com/macros/s/AKfycbwyP9x-mjl2wu8hYRP0Ctyep2BN4VDLmL9czfR-DoH2gi0JebPG-6M2v2SMuVWE0C_yDw/exec"
# url = 'http://127.0.0.1:5000/submit-stock'
# # JSON payload
# payload ={'State': 'Chhattisgarh', 'SS_Name': 'Agarwal Enterprises', 'DB_Name': 'Agarwal corpo.', 'Month': '2025-05', 'stocks': [{'Category': 'Bis', 'Item': 'Product A', 'Mrp': 100, 'Opening': '1238', 'Closing': '90'}, {'Category': 'Bis', 'Item': 'Product B', 'Mrp': 200, 'Opening': '0', 'Closing': '8'}, {'Category': 'Conf', 'Item': 'Product A', 'Mrp': 20, 'Opening': '78', 'Closing': '0'}, {'Category': 'Conf', 'Item': 'Product B', 'Mrp': 20, 'Opening': '8', 'Closing': '0'}]}

In [3]:
# # Send POST request
# response = requests.post(url, json=payload)

# # Print response
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)


## Data Csv to Json

In [4]:
#df = pd.read_csv('Data/db.csv')
df11 = pd.read_excel('Data/sku_master.xlsx',sheet_name='sku_master')

In [5]:
#df11.sort_values(by=['Category','Item_list','Mrp'],inplace=True)
df22 = df11[['Category','Item_list','Mrp']]
df22.to_json('Data/sku_master.json')

In [6]:
# df2 = pd.read_json('Data/sku_master.json') 
# df2

In [7]:
#df = pd.read_excel('Data/db-excel.xlsx',sheet_name='selected2')

In [8]:
df = pd.read_excel('Data/db_master.xlsx',sheet_name='Dist_Detail_Master')

In [9]:
for i in df.select_dtypes(include='object').columns:
    df[i] = df[i].str.strip()
df = df.apply(lambda x: x.str.strip().str.replace(r'\s+', ' ', regex=True) if x.dtype == "object" else x)   

In [10]:
df.columns

Index(['S No', 'STATE NAME', 'ASM Name', 'SS Code', 'SS NAME', 'SS TOWN',
       'Dist. Code', 'Dealer Name (As Per GST)', 'District Name', 'Town Name',
       'Distance KM's From SS Point', 'DB GST Number',
       'DB Contact Person Name', 'DB Contact Person Mob Number',
       'DB Complete Postal Address', 'DB PIN Code', 'DB Email ID', 'DB Status',
       'SR Name', 'Password', 'Bis. Target', 'Conf. Target', 'Noodle  Traget',
       'Total Target', 'Bis. Target  ', 'Bis. Apr', 'Bis. May', 'Bis. Jun',
       'Bis. Jul', 'Bis. Aug', 'Bis. Sep', 'Bis. Oct', 'Bis. Nov', 'Bis. Dec',
       'Bis. Jan', 'Bis. Feb', 'Bis. Mar', 'Conf. Target  ', 'Conf. Apr',
       'Conf. May', 'Conf. Jun', 'Conf. Jul', 'Conf. Aug', 'Conf. Sep',
       'Conf. Oct', 'Conf. Nov', 'Conf. Dec', 'Conf. Jan', 'Conf. Feb',
       'Conf. Mar', 'Juice Target  ', 'Juice Apr', 'Juice May', 'Juice Jun',
       'Juice Jul', 'Juice Aug', 'Juice Sep', 'Juice Oct', 'Juice Nov',
       'Juice Dec', 'Juice Jan', 'Juice Feb', 

In [11]:
df['STATE NAME'] = df['STATE NAME'].str.title().str.strip()
df['ASM Name'] = df['ASM Name'].str.title().str.strip()
df['SS NAME'] = df['SS NAME'].str.title().str.strip()
df['Dealer Name (As Per GST)'] = df['Dealer Name (As Per GST)'].str.title().str.strip()

In [12]:
df['SS Name'] = df['SS NAME']+"|"+df['SS Code']
df['DB Name'] = df['Dealer Name (As Per GST)']+"|"+df['Dist. Code']

In [13]:
df1 = df.rename(columns={
    'STATE NAME':'State',
})

In [14]:
df1 = df1[['State', 'SS Name', 'DB Name', 'Password']]

In [15]:
df1= df1.dropna()

In [16]:
df1['Password'] = np.where(df1['Password']<1000,2025,df1['Password'])

In [17]:
df1['Password'] = df1['Password'].astype(int)
df1['State'] = df1['State'].astype(str).str.strip()
df1['SS Name'] = df1['SS Name'].astype(str).str.strip()
df1['DB Name'] = df1['DB Name'].astype(str).str.strip()

In [18]:
df1.sort_values(by=['State', 'SS Name', 'DB Name', 'Password'],inplace=True)

In [19]:
df1['State'].unique()

array(['Gujarat', 'Haryana', 'Himachal Pradesh', 'Madhya Pradesh',
       'Maharashtra', 'Punjab', 'Rajasthan', 'Uttarakhand'], dtype=object)

In [20]:
df1 = df1[~df1['State'].isin(['Chhattisgarh','Madhya Pradesh'])]

In [21]:
df1.select_dtypes(include='object').columns

Index(['State', 'SS Name', 'DB Name'], dtype='object')

In [22]:
df1[df1.select_dtypes(include='object').columns] = df1.select_dtypes(include='object').apply(lambda x: x.str.strip())


In [23]:
df1.to_csv('Data/master_export.csv',index=False)

In [24]:
# df2 = pd.read_csv('Data/master_export.csv')
# df2['State'] = df1['State'].astype(str).str.strip()
# df2['SS Name'] = df2['SS Name'].astype(str).str.strip()
# df2['DB Name'] = df2['DB Name'].astype(str).str.strip()
# df2 = df2.apply(lambda x: x.str.strip().str.replace(r'\s+', ' ', regex=True) if x.dtype == "object" else x)

In [25]:
# Create nested dictionary with password at DB level
nested = {}

for _, row in df1.iterrows():
    state = row['State']
    ss = row['SS Name']
    db = row['DB Name'] # DB Name
    pwd = row['Password']

    state_dict = nested.setdefault(state, {})
    ss_dict = state_dict.setdefault(ss, {})
    db_dict = ss_dict.setdefault(db, {"password": pwd})


# Save
with open("Data/db.json", "w") as f:
    json.dump(nested, f, indent=4)

In [26]:
#df3 = df2[df2['Password']==3267]

In [27]:
#df3['DB Name'].str.split(' ')

#Mumbai

In [28]:
# mumbai = df1[df1['State'].isin(['Maharashtra'])]

In [29]:
# # Create nested dictionary with password at DB level
# nested = {}

# for _, row in mumbai.iterrows():
#     state = row['State']
#     ss = row['SS Name']
#     db = row['DB Name'] # DB Name
#     pwd = row['Password']

#     state_dict = nested.setdefault(state, {})
#     ss_dict = state_dict.setdefault(ss, {})
#     db_dict = ss_dict.setdefault(db, {"password": pwd})


# # Save
# with open("Data/db.json", "w") as f:
#     json.dump(nested, f, indent=4)